In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import special

In [2]:
def FindSpan(n_inp,degree_inp,u_inp,knot_vector_inp):
    x=knot_vector_inp[degree_inp+1]
    if (u_inp < x):
        return degree_inp
    else:
        for i,pos in enumerate(knot_vector_inp):
            if math.floor(u_inp) == pos:
                return (i)

In [3]:
#The output is stored in N[0],....N[p]
#These are non zero functions in a given span
#i is the span of the u value
#we get N[i-p].........,N[i]
# So if i is 3 then N[3] is stored in N[p] of the output matrix
def BasisFuns(i,u,p,U):
    N =np.zeros(p+1)
    N[0] = 1.0
    left =np.zeros(p+2)
    right =np.zeros(p+2)
    for j in range(1,p+1):
        left[j] = u - U[i+1-j]
        right[j] = U[i+j] - u
        saved= 0.0
        for r in range(0,j):
            temp = N[r]/(right[r+1]+left[j-r])
            N[r] = saved + right[r+1]*temp
            saved = left[j-r]*temp
        N[j] = saved
    return N

In [4]:
# Derivatives are stored in ders[k][j]
# ders[k][j] is the kth derivative of function N_(i-p+j,p)
# If k>p the derivatives are zero.
def DersBasisFuns(i,u,p,n,U):
    #Inititation of dimentions for 2D matrices
    ndu=np.zeros((p+1,p+1))
    ders=np.zeros((p+1,p+1))
    a=np.zeros((2,p+1))
    left =np.zeros(p+2)
    right =np.zeros(p+2)
    
    ndu[0][0]=1.0
    for j in range(1,p+1):
        left[j] = u - U[i+1-j]
        right[j] = U[i+j] - u
        saved=0.0
        for r in range(j):
            #Lower triangle
            ndu[j][r] = right[r+1]+left[j-r]
            temp=ndu[r][j-1]/ndu[j][r]
            #Upper triangle
            ndu[r][j] = saved+(right[r+1]*temp)
            saved=left[j-r]*temp
        ndu[j][j] = saved
    for j in range (p+1): #Load the basis functions
        ders[0][j]=ndu[j][p]
    #This secion computes the derivatives
    for r in range(p+1):
        s1=0
        s2=1 #Alternative rows in array a
        a[0][0] = 1.0
        #Loop to compute kth derivative
        for k in range(1,n+1):
            d=0.0
            rk=r-k
            pk=p-k
            if(r>=k):
                a[s2][0]=a[s1][0]/ndu[pk+1][rk]
                d=a[s2][0]*ndu[rk][pk]
            if(rk>=-1):
                j1=1
            else:
                j1=-rk
            if(r-1<=pk):
                j2=k-1
            else:
                j2=p-r
            for j in range (j1,j2+1):
                a[s2][j] =(a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                d += (a[s2][j]*ndu[rk+j][pk])
            if(r<=pk):
                a[s2][k]=-a[s1][k-1]/ndu[pk+1][r]
                d+=(a[s2][k]*ndu[r][pk])
            ders[k][r]=d
            #Switch rows
            j=s1
            s1=s2
            s2=j
            #Multiply through by the correct factors
    r=p
    for k in range(1,n+1):
        for j in range(p+1):
            ders[k][j] =ders[k][j]* r
        r =r* (p-k)
    return ders

In [8]:
def SurfaceDerivsAlgNURBS(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,1)) #Have to cross check if this is right way of dimensioning SKL
    SKL=np.zeros((20,20,1))
    temp=np.zeros((q+1,1))    # Have to be because W is a scalar unlike P which is vector
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    #print('USPAN',uspan)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    #print('VSPAN',vspan)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning wders with weights equal to 1 here

In [19]:
xi=0.6  #Have to loop over **at the end**
eta=0.6
p=3
q=3
U=np.array([0., 0., 0., 0., 1., 2., 2., 2., 2.])
V=np.array([0., 0., 0., 0., 1., 2., 2., 2., 2.])
P_W=np.array([[[0,0,0,1],[0,0.25,0,1],[0,0.5,0,1],[0,0.75,0,1],[0,1,0,1]],
              [[0.25,0,0,1],[0.25,0.25,0,1],[0.25,0.5,0,1],[0.25,0.75,0,1],[0.25,1,0,1]],
              [[0.5,0,0,1],[0.5,0.25,0,1],[0.5,0.5,0,1],[0.5,0.75,0,1],[0.5,1,0,1]],
              [[0.75,0,0,1],[0.75,0.25,0,1],[0.75,0.5,0,1],[0.75,0.75,0,1],[0.75,1,0,1]],
              [[1.0,0,0,1],[1.0,0.25,0,1],[1.0,0.5,0,1],[1.0,0.75,0,1],[1.0,1,0,1]]])
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
P_1=P_W[:,:,0:3]
W_1=P_W[:,:,3]
d=2

uspan = FindSpan(n,p,xi,U)
BFu = BasisFuns(uspan,xi,p,U)
DBFu = DersBasisFuns(uspan,xi,p,d,U) # Cross check this d

vspan = FindSpan(m,q,eta,V)
BFv = BasisFuns(vspan,eta,q,V)
DBFv = DersBasisFuns(vspan,eta,q,d,V)

#------loop for gauss points-----------#abs    #Have to loop over **at the end**
#Assuming weights as 1 as of now
Den = SurfaceDerivsAlgNURBS(n,p,U,m,q,V,P_1,W_1,xi,eta,d)
Num = BFu[p]*BFv[p]               # a,b control point index
Num_du = DBFu[1][p]*BFv[p]
Num_dv = BFu[p]*DBFv[1][p]
Denom = Den[0][0]
Denom_du = Den[1][0]
Denom_dv = Den[0][1]
Bu = Num_du/Denom - Denom_du*Num/(Denom*Denom)
print(BFu[3])
print(Num,Num_du,Num_dv,Denom,Denom_du,Denom_dv,Bu)






0.054
0.0029159999999999998 0.014580000000000001 0.014580000000000001 [1.] [0.] [0.] [0.01458]
